# Problema delle N-REGINE

Decidiamo di limitare lo spazio degli stati come segue:
   - Consideriamo solo stati, ossia disposizioni delle regine nella scacchiera, nei quali ci sia solo una regina in ogni colonna e in ogni riga.
   - Consideriamo come possibili mosse per passare da uno stato ad uno stato successore lo scambio di due colonne qualsiasi della scacchiera.
   - In tal modo dobbiamo considerare solo gli attacchi sulle diagonali.

Pertanto il problema è così definito:
- Stati: una qualsiasi disposizione di n regine, in modo tale che ci sia una sola regina per colonna e una sola regina per riga.
- Goal State: una qualsiasi disposizione delle n regine sulla scacchiera che non si attaccano a vicenda.
- Funzione successore: un qualsiasi scambio di due colonne della scacchiera.
- Funzione di valutazione: numero di attacchi.
- Test obiettivo: numero di attacchi uguale a zero

In [14]:
import random

def inizializza(sol):

    for c in range(0, DIMENSIONE-1):
        sol = tweak(sol)
    return sol

## TWEAK
La funzione tweak sceglie casualmente due indici x e y nell’encoding, scambiando poi i valori degli elementi corrispondenti

In [15]:
import copy
import numpy as np
def tweak(sol):
    sol_copy = np.copy(sol)

    #scegli random due colonne distinte
    x = random.randint(0, DIMENSIONE-1)
    y = random.randint(0, DIMENSIONE-1)

    while x == y:
        y = random.randint(0, DIMENSIONE-1)

    #scambia le due colonne
    temp = sol_copy[y]
    sol_copy[y]= sol_copy[x]
    sol_copy[x] = temp

    return sol_copy

## FUNZIONE DI VALUTAZIONE DELLO STATO
Tale funzione calcola gli attacchi presenti in un certo stato

In [16]:
def eval_stato(stato):
    # definizione della scacchiera NxN
    board = [ [0] * DIMENSIONE for i in range(DIMENSIONE)]

    #inserimento delle regine('Q') nelle loro posizioni sulla scacchiera
    for i in range(0, DIMENSIONE):
        board[stato[i]][i]='Q'

    #spostamenti possibili sulla scacchiera
    dx = [-1,1,-1,1]
    dy = [-1,1,1,-1]

    conflitti = 0   #inizializziamo il num. di conflitti

    for i in range(0, DIMENSIONE):
        x=stato[i]
        y=i

        #verifica attacchi sulla diagonale
        for j in range(0,4):
            tempx=x
            tempy=y
            while(True):
                tempx = tempx + dx[j]
                tempy = tempy + dy[j]
                if tempx<0 or tempx>=DIMENSIONE or tempy<0 or tempy>=DIMENSIONE:
                    break
                if board[tempx][tempy]=='Q':
                    conflitti = conflitti+1
    return conflitti

## FUNZIONE CHE GENERA IL NEIGHBORHOOD
Tale funzione genera gli stati del vicinato

In [17]:
def genera_successori(stato):
    '''
    GENERA LA LISTA ORDINATA DI SUCCESSORI DI UNO STATO
    :param stato:
    :return:
    '''
    lista = []
    t = len(stato)

    for i in range(0, t-1):
        for j in range(i+1, t):
            buffer = copy.deepcopy(stato)
            temp = buffer[i]
            buffer[i] = buffer[j]
            buffer[j] = temp
            eval_next = eval_stato(buffer)
            lista.append( (buffer, eval_next, (stato[i], stato[j])))

    lista.sort(key=lambda x:x[1])   #ordiniamo i successori in base alla loro valutazione
    return lista

La funzione precedente, per ogni coppia di indici i e j dello
stato, scambia i valori degli elementi corrispondenti. Ad
esempio:    statox = [A, B, C, D, E] DIVENTA stato_successore=[A, D, C, B, E] con cambia a i=B e j=D

## FUNZIONE TABU TEST
Tale funzione controlla se una mossa è ASSENTE nella Tabu List

In [18]:
def tabu_test(sequenza, tabu_list): #è True se una mossa NON è presente
    a, b = sequenza[2]
    if ( (a,b) in tabu_list or (b,a) in tabu_list):
        assente = False
    else:
        assente = True
    return assente

## FUNZIONE STAMPA

In [19]:
def stampa(sol):
    board = [ [0] * DIMENSIONE for i in range(DIMENSIONE)]

    for x in range(0,DIMENSIONE):
        board[sol[x]][x]='Q'
    print('SCACCHIERA')
    for x in range(0, DIMENSIONE):
        for y in range(0,DIMENSIONE):
            if board[x][y]=='Q':
                print("Q  ", end='')
            else:
                print(".  ", end='')
        print("\n")
    print("\n\n")

## ALGORITMO

In [20]:
def tabu_search(tabu_tenure):
    # impostazione stato iniziale
    stato_iniziale = list(x for x in range(DIMENSIONE))
    current = inizializza(stato_iniziale)
    current_eval = eval_stato(current)

    #inizializziamo il best
    best = stato_iniziale
    eval_best = current_eval

    tabu_list = {}
    neighbours = []
    count = 0

    print('STATO INIZIALE(', str(count), '):')
    print(current)
    print('EVAL STATO INIZIALE(', str(count), '):')
    print(current_eval)
    print('----------------------')

    #while not criterio_di_terminazione
    while count < 100 and eval_best!=0:

        lista_successori = genera_successori(current)   #genera i successori (stato, eval_stato, mossa)

        if count == 0:
            l = len(lista_successori)
            print("numero Successori: ", l, '\n')

        #selezione vicinato
        neighbours = list(filter(lambda n: tabu_test(n, tabu_list), lista_successori))  #selezione successori non tabu

        next_state = neighbours[0][0]   #selezione del migliore dei successori
        eval_next_state = neighbours[0][1]
        print('Iterazione: ', count, ':')
        print('Next State: ', eval_next_state)
        delta = eval_best - eval_next_state

        if delta>0:
            best=next_state
            eval_best = eval_next_state

        current = next_state
        current_eval = eval_next_state

        #decremento del tabu_tenure
        for mossa in tabu_list:
            tabu_list[mossa] = tabu_list[mossa] - 1

        #eliminazione elementi con tenure uguale a zero
        tabu_list = {k: tabu_list[k] for k in tabu_list if tabu_list[k] != 0}

        #inserimento della mossa di next in tabu_list
        mossa_next = neighbours[0][2]
        tabu_list[mossa_next] = tabu_tenure

        print('best_eval = ', eval_best)
        print('mossa:', mossa_next)
        print('tabu_list:', tabu_list, '\n')

        count += 1

        print('----------------------')

    return best, eval_best

In [21]:
DIMENSIONE = 30

In [22]:
soluzione, conflitti = tabu_search(3)

STATO INIZIALE( 0 ):
[ 4  0 15  3 19 17  8 24  2  6 10 11  7 21 25 22 16  5 27 18 29  1 12 14
 26 20 23  9 28 13]
EVAL STATO INIZIALE( 0 ):
54
----------------------
numero Successori:  435 

Iterazione:  0 :
Next State:  36
best_eval =  36
mossa: (11, 27)
tabu_list: {(11, 27): 3} 

----------------------
Iterazione:  1 :
Next State:  26
best_eval =  26
mossa: (5, 18)
tabu_list: {(11, 27): 2, (5, 18): 3} 

----------------------
Iterazione:  2 :
Next State:  18
best_eval =  18
mossa: (3, 22)
tabu_list: {(11, 27): 1, (5, 18): 2, (3, 22): 3} 

----------------------
Iterazione:  3 :
Next State:  12
best_eval =  12
mossa: (20, 28)
tabu_list: {(5, 18): 1, (3, 22): 2, (20, 28): 3} 

----------------------
Iterazione:  4 :
Next State:  8
best_eval =  8
mossa: (17, 23)
tabu_list: {(3, 22): 1, (20, 28): 2, (17, 23): 3} 

----------------------
Iterazione:  5 :
Next State:  6
best_eval =  6
mossa: (21, 14)
tabu_list: {(20, 28): 1, (17, 23): 2, (21, 14): 3} 

----------------------
Iterazione:  

In [23]:
soluzione

array([ 4,  0, 15, 22, 19, 23,  8, 24,  2,  6, 10, 27, 20, 14, 25,  3, 26,
       13, 11,  5, 29,  1, 12, 21, 16, 28, 17,  9,  7, 18])

In [24]:
conflitti

0

In [25]:
stampa(soluzione)

SCACCHIERA
.  Q  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  

.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Q  .  .  .  .  .  .  .  .  

.  .  .  .  .  .  .  .  Q  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  

.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Q  .  .  .  .  .  .  .  .  .  .  .  .  .  .  

Q  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  

.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Q  .  .  .  .  .  .  .  .  .  .  

.  .  .  .  .  .  .  .  .  Q  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  

.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Q  .  

.  .  .  .  .  .  Q  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  

.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Q  .  .  

.  .  .  .  .  .  .  .  .  .  Q  .  .  .  .  .  .  .  .  .  .  .  .  